In [1]:
from info_titul import DatosWeb
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import urllib3


urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


### Proving the library step-by-step

#### 1. 'get_basic_data' function

In [42]:
url_base = "https://www.educacion.gob.es/ruct/solicitud/detalles.action?cod=codigoin&sit=A&actual=menu.solicitud.basicos"
variable = DatosWeb.get_basic_data(url_base, "25000302008052801")

In [43]:
variable

,Nombre,Conjunto,Rama,Habilita,Vinculacion,Codigo de Agencia
25000302008052801,Graduado o Graduada en Administración de Empre...,No,Ciencias Sociales y Jurídicas,No,No,Agencia Nacional de Evaluación de la Calidad y...


#### 2. 'get_year function

In [5]:
# La URL base corresponde a la pantalla de 'Calendario de Implantación'.
url_base = "https://www.educacion.gob.es/ruct/solicitud/calendarioImplantacion!cronograma?actual=menu.solicitud.calendarioImplantacion.cronograma&cod=codigoin"
variable2 = DatosWeb.get_year(url_base, "25006612023052201")

In [6]:
variable2

{'25006612023052201': '2009'}

#### 3. 'get_competencies' function

In [44]:
url_base = "https://www.educacion.gob.es/ruct/solicitud/competencias?actual=menu.solicitud.competencias.palabratipocomp&tipo=tipodecomp&cod=codigoin"
variable3 = DatosWeb.get_competencies(url_base, "25006612023052201","Compentencias Generales")

In [45]:
variable3

{'25006612023052201': ['Que los estudiantes hayan demostrado poseer y comprender conocimientos en un área de estudio que parte de la base de la educación secundaria general, y se suele encontrar a un nivel que, si bien se apoya en libros de texto avanzados, incluye también algunos aspectos que implican conocimientos procedentes de la vanguardia de su campo de estudio',
  'Que los estudiantes sepan aplicar sus conocimientos a su trabajo o vocación de una forma profesional y posean las competencias que suelen demostrarse por medio de la elaboración y defensa de argumentos y la resolución de problemas dentro de su área de estudio',
  'Que los estudiantes tengan la capacidad de reunir e interpretar datos relevantes (normalmente dentro de su área de estudio) para emitir juicios que incluyan una reflexión sobre temas relevantes de índole social, científica o ética',
  'Que los estudiantes puedan transmitir información, ideas, problemas y soluciones a un público tanto especializado como no es

#### 4. 'get_info' function

In [47]:
url_base_sistemasdeformacion = "https://www.educacion.gob.es/ruct/solicitud/sistemas?actual=menu.solicitud.planificacion.sistemas&cod=codigoin"
url_base_actividades = "https://www.educacion.gob.es/ruct/solicitud/actividades?actual=menu.solicitud.planificacion.actividades&cod=codigoin"
url_base_metodologias = "https://www.educacion.gob.es/ruct/solicitud/metodologias?actual=menu.solicitud.planificacion.metodologias&cod=codigoin"

variable4 = DatosWeb.get_info(url_base_sistemasdeformacion, "25006612023052201")
variable5 = DatosWeb.get_info(url_base_actividades, "25006612023052201")
variable6 = DatosWeb.get_info(url_base_metodologias, "25006612023052201")

In [48]:
variable4

{'25006612023052201': ['Examen teórico-práctico',
  'Evaluación continua: resolución oral y escrita de casos clínicos y problemas, demostraciones prácticas en clase, adquisición de competencias actitudinales, portafolios, diarios reflexivos, etc.',
  'Presentación, exposición y defensa de trabajos.',
  'Evaluación de memorias de prácticas clínicas',
  'Informes de tutores académicos y profesionales']}

In [49]:
variable5

{'25006612023052201': ['Presencial: Clases Teóricas',
  'Presencial: Clases Prácticas',
  'Presencial: Seminarios y Talleres',
  'Presencial: Tutorías Programadas y a demanda',
  'Presencial: Prácticas Clínicas',
  'Presencial: Exámenes y otras actividades de Evaluación',
  'No presencial: Estudio personal, perfeccionamiento en habilidades y destrezas, preparación de trabajos individuales y en grupo, búsqueda y gestión autónoma de información, preparación de exámenes y otras actividades de evaluación.']}

In [50]:
variable6

{'25006612023052201': ['Lección magistral',
  'Estudio de casos',
  'Estudio de problemas',
  'Escenificación y representación de situaciones clínicas',
  'Exposición y presentación de trabajos',
  'Trabajo cooperativo',
  'Demostraciones activas',
  'Formación mediante monitores',
  'Portafolio, diarios reflexivos y memorias de prácticas',
  'Prácticas de disección anatómica',
  'Debate y puesta en común',
  'Búsqueda de información y documentación',
  'Simulación Práctica']}

#### 5. 'get_status' function

In [52]:
base_url = "https://www.educacion.gob.es/ruct/listaestudios?codigoRama=&actual=estudios&d-1335801-p=codigotablas&codigoEstado=&buscarHistorico=N&descripcionEstudio=&codigoTipo=&codigoSubTipo=&codigoEstudio=&situacion=&ambito=&action:listaestudios=Consultar&codigoUniversidad=universidad&consulta=1"

In [60]:
variable7 = DatosWeb.get_status(base_url, "4316264" , "036", 10, "Estado")

In [61]:
variable7

{'4316264': ['Publicado en B.O.E.  (TITULACIÓN RENOVADA)']}

In [ ]:
# formato base de datos
def extract_first_level(url, id):
    """
    Extracts data from the given URL and ID.
    
    Parameters:
    - url (str): The URL of the webpage.
    - id (str): The ID used in the URL.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the extracted data.
    """
    response = requests.get(url.replace('codigoin', id), verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        data = []
        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            materia_url = row.find('a', class_='modificar')['href']
            materia_url = "https://www.educacion.gob.es" + materia_url
            materia_response = requests.get(materia_url, verify=False)
            if materia_response.status_code == 200:
                materia_soup = BeautifulSoup(materia_response.text, 'html.parser')
                materia_table_rows = materia_soup.select('#materia tbody tr')
                for materia_row in materia_table_rows:
                    asignatura = materia_row.find_all('td')[1].get_text(strip=True)
                    data.append({'Denominación': denominacion, 'Asignatura': asignatura})

        df = pd.DataFrame(data)
        return df
    else:
        print("Error: Unable to fetch data from the URL")
        return None

    
url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df

In [24]:
def extract_first_level(url, id):
    """
    Extracts data from the given URL and ID.
    
    Parameters:
    - url (str): The URL of the webpage.
    - id (str): The ID used in the URL.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the extracted data.
    """
    response = requests.get(url.replace('codigoin', id), verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        data = []
        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            materia_url = row.find('a', class_='modificar')['href']
            materia_url = "https://www.educacion.gob.es" + materia_url
            materia_response = requests.get(materia_url, verify=False)
            if materia_response.status_code == 200:
                materia_soup = BeautifulSoup(materia_response.text, 'html.parser')
                materia_table_rows = materia_soup.select('#materia tbody tr')
                for materia_row in materia_table_rows:
                    asignatura = materia_row.find_all('td')[1].get_text(strip=True)
                    asignatura_url = "https://www.educacion.gob.es" + materia_row.find('a')['href']
                    asignatura_response = requests.get(asignatura_url, verify=False)
                    data.append({'Denominación': denominacion, 'Asignatura': asignatura, 'URL': asignatura_url})

        df = pd.DataFrame(data)
        return df
    else:
        print("Error: Unable to fetch data from the URL")
        return None

url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df


,Denominación,Asignatura,URL
0,Formación básica en ciencias sociales/\ Basic ...,Introducción a la administración de empresas /...,https://www.educacion.gob.es/ruct/solicitud/da...
1,Formación básica en ciencias sociales/\ Basic ...,Introducción a la economía / Introduction to e...,https://www.educacion.gob.es/ruct/solicitud/da...
2,Formación básica en ciencias sociales/\ Basic ...,Introducción a la estadística / Introduction t...,https://www.educacion.gob.es/ruct/solicitud/da...
3,Formación básica en ciencias sociales/\ Basic ...,Introducción al derecho / Introduction to Law,https://www.educacion.gob.es/ruct/solicitud/da...
4,Formación básica en ciencias sociales/\ Basic ...,Historia económica / Economic history,https://www.educacion.gob.es/ruct/solicitud/da...
5,Formación básica en ciencias sociales/\ Basic ...,Destrezas y habilidades/ Skills and abilities,https://www.educacion.gob.es/ruct/solicitud/da...


In [35]:
import time

def extract_first_level(url, id):
    response = requests.get(url.replace('codigoin', id), verify=False)
    
    # ¿hay módulos de primer nivel?
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            # página con los módulos de cada denominación
            modulo_url = row.find('a', class_='modificar')['href']
            modulo_url = "https://www.educacion.gob.es" + modulo_url
            print(modulo_url)
            time.sleep(5)  
            modulo_response = requests.get(modulo_url, verify=False)
            
            # sacamos la info del módulo
            if modulo_response.status_code == 200:
                modulo_soup = BeautifulSoup(modulo_response.text, 'html.parser')
                modulo_table_rows = modulo_soup.select('#materia tbody tr')
                
                for modulo_row in modulo_table_rows:
                    asignatura = modulo_row.find_all('td')[1].get_text(strip=True)
                    # sacamos las 
                    asignatura_url = "https://www.educacion.gob.es" + modulo_row.find('a')['href']
                    print(asignatura_url)
                    """
                    asignatura_response = requests.get(asignatura_url, verify=False)
                    asignatura_soup = BeautifulSoup(asignatura_response.text, 'html.parser')

                    # datos modulo: carácter, ects, unidad temporal
                    datos_materia_basicos_div = asignatura_soup.find('div', {'id': 'datosMateria_basicos'})
                    if datos_materia_basicos_div:
                        
                        ects_materia_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsMateria'})
                        if ects_materia_input:
                                ects_basica = ects_materia_input['value']
                                ects_obligatoria = 0
                                ects_optativa = 0
                        else:
                                ects_basica_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsBasica'})
                                ects_obligatoria_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsObligatoria'})
                                ects_optativa_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsOptativa'})
                                ects_basica = ects_basica_input['value'] if ects_basica_input else 0
                                ects_obligatoria = ects_obligatoria_input['value'] if ects_obligatoria_input else 0
                                ects_optativa = ects_optativa_input['value'] if ects_optativa_input else 0
                            
                        caracter_element = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_caracter_codigo'})
                        caracter = caracter_element.get('value','No encontrado')

                        unidad_temporal_element = datos_materia_basicos_div.find('input', {'id': 'datosMateria_unidadTemporal'})
                        unidad_temporal = unidad_temporal_element.get('value', 'No encontrado')
                        
                        lenguas_table = asignatura_soup.find('table', {'id': 'lengua'})
                        if lenguas_table:
                            lenguas = [lenguas_td.get_text(strip=True) for lenguas_td in lenguas_table.select('tbody td')]
                            lenguas_str = ', '.join(lenguas)
                        else:
                            lenguas_str = 'No encontrado'
                            
                        asignaturas_table = asignatura_soup.find('table', {'id': 'asignatura'})
                        if asignaturas_table:
                            asignaturas_rows = asignaturas_table.select('tbody tr')
                            asignaturas_data = []
                            for row in asignaturas_rows:
                                asignatura_id = row.find_all('td')[0].get_text(strip=True)
                                asignatura_nombre = row.find_all('td')[1].get_text(strip=True)
                                print(asignatura_nombre)
                                asignatura_url = "https://www.educacion.gob.es" + row.find('a', class_='modificar')['href']
                                asignaturas_data.append((asignatura_id, asignatura_nombre, asignatura_url))
                            print(asignaturas_data)
                        else:
                            asignaturas_str = 'No encontrado'
                  """      

        return 
        
    else:
        print("Error: Unable to fetch data from the URL")
        return None


In [36]:
url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df

https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=1&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=2&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&codMateria=1&actual=menu.solicitud.planificacion.materias.datos
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&codMateria=2&actual=menu.solicitud.planificacion.materias.datos
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5